# ディープラーニングを数学から理解する

サンプルnotebook

In [0]:
# 必要ライブラリの宣言
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# PDF出力用
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

In [0]:
# 学習データの設定 (numpy二次元配列)
data = np.array([[166,58.7], [176,75.7], 
  [171,62.1], [173,70.4], [169,60.1]])

# データの確認
print(data)

In [0]:
# 散布図で表示 (図)
plt.scatter(data[:,0], data[:,1], s=40, c='b')
plt.grid()
plt.show()

In [0]:
# 身長、体重の平均値の計算
mean = data.mean(axis=0)
print(mean)

In [0]:
mean2 = data.mean(axis=1)
print(mean2)

In [0]:
# 元データの平行移動
data1 = data - mean
print(data1)

In [0]:
# 平行移動後の散布図表示 (図1-8)
plt.scatter(data1[:,0], data1[:,1], s=40, c='b')
plt.plot([-6,6],[0,0], lw=2, c='k')
plt.plot([0,0],[-8,12],lw=2,c='k')
plt.xlim(-6,6)
plt.ylim(-8,12)
plt.grid()
plt.show()

In [0]:
# 入力データの抽出
# ダミー変数追加のため、
# 1次元でなく5x1の2次元データにします。
x1 = data1[:,0:1]
print(x1)
print(x1.shape)

In [0]:
# ダミー変数を追加
x = np.insert(x1, 0, 1.0, axis=1)
print('ダミー変数追加後')
print(x)
print(x.shape)

In [0]:
# 正解データ
yt = data1[:,1]
print(yt)

In [0]:
# 予測関数 (1, x)の値から予測値ypを計算する
def pred(x, w):
    return(x @ w)

In [0]:
# 重みベクトルの初期値 (勝手な値を設定)
w = np.array([2, -1])
yp = pred(x, w)
print(yp)

In [0]:
# 損失関数の計算
yd = yp - yt
L = np.mean(yd ** 2) / 2
print(L)

In [0]:
# 初期化処理

# データ系列総数 (=5)
M  = x.shape[0]

# 入力データ次元数(ダミー変数を含む) (=2)
D = x.shape[1]

# 繰り返し回数
iters = 200

# 学習率
alpha = 0.01

# 重みベクトルの初期値 (勝手な値を設定)
w = np.array([2, -1])

# 評価結果記録用 (損失関数値, w0, w1を記録)
history = np.zeros((0,4))

In [0]:
# 繰り返しループ
for k in range(iters):
    
    # 予測値の計算
    yp = pred(x, w)
    
    # 誤差の計算
    yd = yp - yt
    
    # 勾配降下法の実装
    w = w - alpha * (x.T @ yd) / M
    
    # 学習曲線描画用データの計算、保存

    # 重みベクトル値の取得
    w0 = float(w[0])
    w1 = float(w[1])

    # 損失関数値の計算
    L = np.mean(yd ** 2) / 2
    
    # 計算結果の記録
    line = np.array([k, L, w0, w1])
    history = np.vstack((history, line))
    
    # 画面表示
    print( "iter = %d  L = %f w0 = %f w1 = %f" 
          % (k, L, w0, w1))

In [0]:
np.set_printoptions(precision=3)
yp = pred(x, w)
print('最終的な予測値')
print(yp)
print('正解値')
print(yt)

In [0]:
# 学習曲線の表示
plt.plot(history[:,0], history[:,1])
plt.grid()
plt.show()

In [0]:
# 散布図と回帰直線を同時表示

X = np.array([[1, -6],[1, 6]])
Y = pred(X, w)

plt.scatter(data1[:,0], data1[:,1], s=40, c='b')
plt.plot([-6, 6], Y, lw=2, c='k')
plt.plot([-6,6],[0,0], lw=1, c='k')
plt.plot([0,0],[-8,12],lw=1,c='k')
plt.xlim(-6,6)
plt.ylim(-8,12)
plt.grid()
plt.show()

In [0]:
# アニメーション表示　初期化
from matplotlib import animation, rc
from IPython.display import HTML

# 初期画面の描画
fig, ax = plt.subplots()

# x軸、y軸の範囲
ax.set_xlim(( -6, 6))
ax.set_ylim((-8, 12))

# グラフ描画
ax.plot([-6,6],[0,0], lw=2, c='k')
ax.plot([0,0],[-8,12],lw=2,c='k')
ax.grid()
ax.scatter(data1[:,0], data1[:,1], s=40, c='b')

# グラフの線は枠だけ用意しておきます。
line, = ax.plot([], [], lw=2)

In [0]:
X0 = np.array([-6,6])
X1 = np.array([[1, -6],[1, 6]])

# 初期化関数
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    l = history[i,:]
    W = l[2:]
    L = float(l[1])
    w0 = float(l[2])
    w1 = float(l[3])
    Y = pred(X1, W)
    line.set_data(X0, Y)
    title = 'i = %3d  w0 = %6.3f w1 = %6.3f L = %6.3f' % (i, w0, w1, L)
    ax.set_title(title)
    return (line,)

# アニメーションの定義
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=100, blit=True, repeat=True)

# アニメーションの描画
HTML(anim.to_html5_video())

In [0]:
# Scikit-learnのライブラリを使って線形回帰モデルを作成

from sklearn import linear_model
clf = linear_model.LinearRegression()

# 予測モデルを作成
clf.fit(x1, yt)

# 切片と傾きの取得
W0 = clf.intercept_
W1 = clf.coef_
print('パラメータ値比較')
print('実習で求めた結果 w0 = %6.2f w1 = %6.2f' % (w0, w1))
print('ライブラリの結果 w0 = %6.2f w1 = %6.2f' % (W0, W1))